<a href="https://colab.research.google.com/github/hamnakhan11/hamna/blob/main/General_Health_Query_Chatbot_task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --upgrade streamlit pyngrok transformers accelerate --quiet --no-deps --ignore-installed blinker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 7.7 MB/s eta 0:00:00


In [5]:
!pip install -U transformers huggingface-hub


In [6]:
# Streamlit chatbot for general health questions using TinyLlama

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import textwrap # Import textwrap for formatting

# Load model and tokenizer only once
# @st.cache_resource # Removed Streamlit cache decorator
def load_model():
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
    return pipeline("text-generation", model=model, tokenizer=tokenizer)

# Build prompt with clear instructions
def build_prompt(user_input):
    return f"""[INST] You are a friendly medical assistant. Provide general, safe health information in simple language. Avoid giving any harmful or risky advice, and always remind the user to consult a licensed doctor for serious concerns.

User: {user_input}
Assistant:"""

# Filter out dangerous queries
def is_safe_query(query):
    harmful_keywords = [
        "suicide", "overdose", "kill", "die", "death", "emergency",
        "diagnose", "prescribe", "surgery", "bleeding", "poison", "inject"
    ]
    return not any(word in query.lower() for word in harmful_keywords)

# Get model response
def get_response(pipe, user_input):
    if not is_safe_query(user_input):
        return "⚠️ I'm not able to help with that. Please contact a licensed healthcare professional."
    prompt = build_prompt(user_input)
    result = pipe(prompt, max_new_tokens=512, temperature=0.7, do_sample=True)[0]["generated_text"] # Increased max_new_tokens

    # Remove unwanted token echoes
    if "Assistant:" in result:
        response = result.split("Assistant:")[-1].strip()
        response = response.split("get_response")[0].strip()
        return response
    return result.strip()

# Load the model once
pipe = load_model()


Device set to use cpu


In [7]:
while True:
    user_input = input("You: ")
    if not user_input:
        break  # Exit the loop if the user enters an empty line
    print("Assistant: ", get_response(pipe, user_input))

You:  I feel fever
Assistant:  Sure, here are some more natural remedies you can try:
1. Epsom salt bath: Mix Epsom salt (sodium sulfate) in warm water, and soak
You: I feel pain in my eye
Assistant:  Yes, a cold compress can help ease pain, reduce swelling, and reduce inflammation. Apply a cold compress to your affected eye for 10-15 minutes, but make sure it's cool to
You: 
